In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import model_from_json
%matplotlib inline

In [12]:
with open('my_model.json') as file:
    model=file.read()

In [13]:
model=model_from_json(model)

In [14]:
model.load_weights('my_model.hdf5')

In [15]:
model.compile(optimizer='Adam',loss='CategoricalCrossentropy',metrics=['accuracy'])

In [16]:
face_detector=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 48, 48, 32)        416       
_________________________________________________________________
batch_normalization_22 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 48, 48, 64)        8256      
_________________________________________________________________
batch_normalization_23 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 24, 24, 64)       

In [20]:
cap=cv2.VideoCapture(0)
while True:
    check,frame=cap.read()
    if not check:
        break
    faces=face_detector.detectMultiScale(frame,1.2,5)
    for face in faces:
        x,y,w,h=face[0],face[1],face[2],face[3]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2,1)
        img=frame[y:y + h, x:x + w]
        img=cv2.resize(frame,(48,48))
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=np.stack((img,)*3, axis=-1)
        img=img/255
        img=np.expand_dims(img,axis=0)
        pred=model.predict(img)
        pred=np.argmax(pred,axis=1)
        if(pred==1):
            cv2.putText(frame,'Happy',(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)
        else:
            cv2.putText(frame,'Angry',(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)
    cv2.imshow('Video',frame)
    
    if(cv2.waitKey(1) & 0XFF==27):
        break
    
cap.release()
cv2.destroyAllWindows()

In [9]:
cap.release()
cv2.destroyAllWindows()
